In [1]:
# 1.Kadamba Transport Corporation Limited (KTCL)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    # if index >= len(routes):
    #     print(f"Route index {index} out of range after refresh.")
    #     break

    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

cursor = connection.cursor()

create_table_query = """
        CREATE TABLE redbus.bus_data_info (
            id INT AUTO_INCREMENT PRIMARY KEY,
            route_name VARCHAR(255),
            route_link VARCHAR(255),
            busname VARCHAR(255),
            bustype VARCHAR(255),
            departing_time VARCHAR(255),
            duration VARCHAR(255),
            Date VARCHAR(255),
            reaching_time VARCHAR(255),
            price VARCHAR(255),
            seats_available VARCHAR(255)
        )
        """ 
cursor.execute(create_table_query)

print("table created")


insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

bus_data = bus_lst

for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()
connection.close()

0
Processing route: Pune-to-Goa
1
Processing route: Goa-to-Pune
2
Processing route: Mumbai-to-Goa
3
Processing route: Goa-to-Mumbai
4
Processing route: Pandharpur-to-Goa
5
Processing route: Bangalore-to-Goa
6
Processing route: Goa-to-Pandharpur
7
Processing route: Belagavi-to-Goa
8
Processing route: Goa-to-Bangalore
9
Processing route: Solapur-to-Goa
[{'routename': 'Pune-to-Goa', 'routelink': 'https://www.redbus.in/bus-tickets/Pune-to-Goa?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'Kadamba Transport Corporation Limited (KTCL) - 89821', 'bus_type': 'A/C Sleeper (2+1)', 'dep_time': '19:15', 'd_duration': '10h 35m', 'arr_time': '05:50', 'f_fare': 'INR 800', 's_seat': '17 Seats available'}, {'routename': 'Pune-to-Goa', 'routelink': 'https://www.redbus.in/bus-tickets/Pune-to-Goa?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_nam

In [3]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
# with col3:
#     selected_departing_time = st.selectbox("Select Bus Name", sorted(set(df["departing_time"])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
# if selected_departing_time:
#     filtered_data = filtered_data[filtered_data["departing_time"] == selected_route_name]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)


    # def make_clickable(link):
    #     return f'<a href="{link}" target="_blank">Link</a>'

    # filtered_data['route_link'] = filtered_data['route_link'].apply(make_clickable)

    # Display DataFrame as HTML table
    # st.write("""
    #     <style>
    #         .dataframe {
    #             width: 100%;
    #             border-collapse: collapse;
    #         }
    #         .dataframe th, .dataframe td {
    #             padding: 8px 12px;
    #             border: 1px solid #ddd;
    #             text-align: left;
    #         }
    #         .dataframe th {
    #             background-color: #f2f2f2;
    #         }
    #     </style>
    # """, unsafe_allow_html=True)

    # st.write(filtered_data.to_html(escape=False, index=False), unsafe_allow_html=True)
    
else:
    st.warning("No data found for selected filters.")

2024-07-26 09:39:07.036 
  command:

    streamlit run /opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


          route_name                                         route_link  \
0        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...              ...                                                ...   
1042  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1043  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1044  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1045  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1046  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   

                                                busname  \
0     Kadamba Transport Corporation Limi

In [4]:
!pip install streamlit 

In [5]:
#2.TSRTC Online Bus Ticket Booking

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    # if index >= len(routes):
    #     print(f"Route index {index} out of range after refresh.")
    #
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=401&toCityId=124&fromCityName=Khammam&toCityName=Hyderabad&busType=Any&onward={travel_date}")
    route_url =(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=401&toCityId=124&fromCityName=Khammam&toCityName=Hyderabad&busType=Any&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

cursor = connection.cursor()

#create_table_query = """
        #CREATE TABLE redbus.bus_data_info (
            #id INT AUTO_INCREMENT PRIMARY KEY,
            #route_name VARCHAR(255),
            #route_link VARCHAR(255),
            #busname VARCHAR(255),
            #bustype VARCHAR(255),
            #departing_time VARCHAR(255),
            #duration VARCHAR(255),
            #Date VARCHAR(255),
            #reaching_time VARCHAR(255),
            #star_rating VARCHAR(255),
            #price VARCHAR(255),
            #eats_available VARCHAR(255)
#cursor.execute(create_table_query)

#print("table created")


insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

bus_data = bus_lst

for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()
connection.close()


0
Processing route: Khammam-to-Hyderabad
1
Processing route: Hyderabad-to-Vijayawada
2
Processing route: Hyderabad-to-Khammam
3
Processing route: Hyderabad-to-Srisailam
4
Processing route: Karimnagar-to-Hyderabad
5
Processing route: Hyderabad-to-Nirmal
6
Processing route: Hyderabad-to-Mancherial
7
Processing route: Hyderabad-to-Adilabad
8
Processing route: Hyderabad-to-Karimnagar
9
Processing route: Kothagudem-to-Hyderabad
[]
data inserted


In [7]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
# with col3:
#     selected_departing_time = st.selectbox("Select Bus Name", sorted(set(df["departing_time"])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
# if selected_departing_time:
#     filtered_data = filtered_data[filtered_data["departing_time"] == selected_route_name]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)


    # def make_clickable(link):
    #     return f'<a href="{link}" target="_blank">Link</a>'

    # filtered_data['route_link'] = filtered_data['route_link'].apply(make_clickable)

    # Display DataFrame as HTML table
    # st.write("""
    #     <style>
    #         .dataframe {
    #             width: 100%;
    #             border-collapse: collapse;
    #         }
    #         .dataframe th, .dataframe td {
    #             padding: 8px 12px;
    #             border: 1px solid #ddd;
    #             text-align: left;
    #         }
    #         .dataframe th {
    #             background-color: #f2f2f2;
    #         }
    #     </style>
    # """, unsafe_allow_html=True)

    # st.write(filtered_data.to_html(escape=False, index=False), unsafe_allow_html=True)
    
else:
    st.warning("No data found for selected filters.")

          route_name                                         route_link  \
0        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4        Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...              ...                                                ...   
1042  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1043  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1044  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1045  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   
1046  Solapur-to-Goa  https://www.redbus.in/bus-tickets/Solapur-to-G...   

                                                busname  \
0     Kadamba Transport Corporation Limi

In [8]:
# 3.APSRTC

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    # if index >= len(routes):
    #     print(f"Route index {index} out of range after refresh.")
    #     break

    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad?fromCityId=134&toCityId=124&fromCityName=Vijayawada&toCityName=Hyderabad&busType=Any&onward=22-Jul-2024")
    route_url = (f"https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad?fromCityId=134&toCityId=124&fromCityName=Vijayawada&toCityName=Hyderabad&busType=Any&onward=22-Jul-2024")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Vijayawada-to-Hyderabad
1
Processing route: Hyderabad-to-Vijayawada
2
Processing route: Kakinada-to-Visakhapatnam
3
Processing route: Visakhapatnam-to-Kakinada
4
Processing route: Chittoor-(Andhra-Pradesh)-to-Bangalore
5
Processing route: Kadapa-to-Bangalore
6
Processing route: Anantapur-(andhra-pradesh)-to-Bangalore
7
Processing route: Tirupati-to-Bangalore
8
Processing route: Visakhapatnam-to-Vijayawada
9
Processing route: Ongole-to-Hyderabad
[{'routename': 'Vijayawada-to-Hyderabad', 'routelink': 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad?fromCityId=134&toCityId=124&fromCityName=Vijayawada&toCityName=Hyderabad&busType=Any&onward=22-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'IntrCity SmartBus', 'bus_type': 'A/C Seater / Sleeper (2+1)', 'dep_time': '23:45', 'd_duration': '06h 15m', 'arr_time': '06:00', 'f_fare': 'INR 535', 's_seat': '34 Seats available'}, {'routename': 'Vijayawada-to-Hyderabad', 'routelink': 'https://www.redbus.in/bus-tickets/v

True

In [10]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                       route_name  \
0                     Goa-to-Pune   
1                     Goa-to-Pune   
2                     Goa-to-Pune   
3                     Goa-to-Pune   
4                     Goa-to-Pune   
...                           ...   
1072  Visakhapatnam-to-Vijayawada   
1073  Visakhapatnam-to-Vijayawada   
1074          Ongole-to-Hyderabad   
1075          Ongole-to-Hyderabad   
1076          Ongole-to-Hyderabad   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                                                 ...   
1072  https://www.redbus.in/bus-tickets/vijayawada-t...   
1073  https://www.redbus.in/bus-tickets/vijayawada-t...   
1074  https://www.redbus

In [11]:
#4.RSRTC

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/jodhpur-to-ajmer?fromCityId=1169&toCityId=808&fromCityName=Jodhpur&toCityName=Ajmer&busType=Any&onward=22-Jul-2024")
    route_url = (f"https://www.redbus.in/bus-tickets/jodhpur-to-ajmer?fromCityId=1169&toCityId=808&fromCityName=Jodhpur&toCityName=Ajmer&busType=Any&onward=22-Jul-2024")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Jodhpur-to-Ajmer
1
Processing route: Beawar-(Rajasthan)-to-Jaipur-(Rajasthan)
2
Processing route: Udaipur-to-Jodhpur
3
Processing route: Jaipur-(Rajasthan)-to-Jodhpur
4
Processing route: Sikar-to-Jaipur-(Rajasthan)
5
Processing route: Kishangarh-to-Jaipur-(Rajasthan)
6
Processing route: Aligarh-(uttar-pradesh)-to-Jaipur-(Rajasthan)
7
Processing route: Jodhpur-to-Beawar-(Rajasthan)
8
Processing route: Kota(Rajasthan)-to-Jaipur-(Rajasthan)
9
Processing route: Jaipur-(Rajasthan)-to-Aligarh-(uttar-pradesh)
[]
data inserted


True

In [13]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                       route_name  \
0                     Goa-to-Pune   
1                     Goa-to-Pune   
2                     Goa-to-Pune   
3                     Goa-to-Pune   
4                     Goa-to-Pune   
...                           ...   
1072  Visakhapatnam-to-Vijayawada   
1073  Visakhapatnam-to-Vijayawada   
1074          Ongole-to-Hyderabad   
1075          Ongole-to-Hyderabad   
1076          Ongole-to-Hyderabad   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                                                 ...   
1072  https://www.redbus.in/bus-tickets/vijayawada-t...   
1073  https://www.redbus.in/bus-tickets/vijayawada-t...   
1074  https://www.redbus

In [14]:
#5.Kerala RTC Online Ticket Booking

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=122&toCityId=74661&fromCityName=Bangalore&toCityName=Kozhikode&busType=Any&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=122&toCityId=74661&fromCityName=Bangalore&toCityName=Kozhikode&busType=Any&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Bangalore-to-Kozhikode
1
Processing route: Kozhikode-to-Ernakulam
2
Processing route: Kozhikode-to-Bangalore
3
Processing route: Ernakulam-to-Kozhikode
4
Processing route: Kozhikode-to-Mysore
5
Processing route: Kozhikode-to-Thiruvananthapuram
6
Processing route: Bangalore-to-Kalpetta-(kerala)
7
Processing route: Mysore-to-Kozhikode
8
Processing route: Kalpetta-(kerala)-to-Bangalore
9
Processing route: Kozhikode-to-Thrissur
[{'routename': 'Bangalore-to-Kozhikode', 'routelink': 'https://www.redbus.in/bus-tickets/Bangalore-to-Kozhikode?fromCityId=122&toCityId=74661&fromCityName=Bangalore&toCityName=Kozhikode&busType=Any&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'MMK Travels', 'bus_type': 'VE A/C Seater / Sleeper (2+1)', 'dep_time': '22:30', 'd_duration': '09h 00m', 'arr_time': '07:30', 'f_fare': 'INR 999', 's_seat': '25 Seats available'}, {'routename': 'Bangalore-to-Kozhikode', 'routelink': 'https://www.redbus.in/bus-tickets/Bangalore-to-Kozhikode?from

True

In [16]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                 route_name  \
0               Goa-to-Pune   
1               Goa-to-Pune   
2               Goa-to-Pune   
3               Goa-to-Pune   
4               Goa-to-Pune   
...                     ...   
1122  Kozhikode-to-Thrissur   
1123  Kozhikode-to-Thrissur   
1124  Kozhikode-to-Thrissur   
1125  Kozhikode-to-Thrissur   
1126  Kozhikode-to-Thrissur   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                                                 ...   
1122  https://www.redbus.in/bus-tickets/Kozhikode-to...   
1123  https://www.redbus.in/bus-tickets/Kozhikode-to...   
1124  https://www.redbus.in/bus-tickets/Kozhikode-to...   
1125  https://www.redbus.in/bus-ticke

In [17]:
#6.NORTH BENGAL STATE TRANSPORT CORPORATION

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/north-bengal-state-transport-corporation')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/north-bengal-state-transport-corporation')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=74820&toCityId=74694&fromCityName=Kolkata&toCityName=Siliguri&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=74820&toCityId=74694&fromCityName=Kolkata&toCityName=Siliguri&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Kolkata-to-Siliguri
1
Processing route: Siliguri-to-Kolkata
2
Processing route: Siliguri-to-Darjeeling
3
Processing route: Kolkata-to-Raiganj
4
Processing route: Raiganj-to-Kolkata
5
Processing route: Kolkata-to-Malda
6
Processing route: Cooch-Behar-(West-Bengal)-to-Berhampore-(West-Bengal)
7
Processing route: Kolkata-to-Cooch-Behar-(West-Bengal)
8
Processing route: Malda-to-Kolkata
9
Processing route: Berhampore-(West-Bengal)-to-Cooch-Behar-(West-Bengal)
[{'routename': 'Kolkata-to-Siliguri', 'routelink': 'https://www.redbus.in/bus-tickets/Kolkata-to-Siliguri?fromCityId=74820&toCityId=74694&fromCityName=Kolkata&toCityName=Siliguri&busType=Any&srcCountry=IND&destCountry=IND&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'SBSTC-KOLKATA - SILIGURI - VIA - MALDA - 20:00 (BEL DEPO - 3805', 'bus_type': 'Express (Non AC Seater 2+2)', 'dep_time': '20:00', 'd_duration': '13h 00m', 'arr_time': '09:00', 'f_fare': 'INR 500', 's_seat': '29 Seats available'}, {'routena

True

In [19]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                                             route_name  \
0                                           Goa-to-Pune   
1                                           Goa-to-Pune   
2                                           Goa-to-Pune   
3                                           Goa-to-Pune   
4                                           Goa-to-Pune   
...                                                 ...   
1172  Berhampore-(West-Bengal)-to-Cooch-Behar-(West-...   
1173  Berhampore-(West-Bengal)-to-Cooch-Behar-(West-...   
1174  Berhampore-(West-Bengal)-to-Cooch-Behar-(West-...   
1175  Berhampore-(West-Bengal)-to-Cooch-Behar-(West-...   
1176  Berhampore-(West-Bengal)-to-Cooch-Behar-(West-...   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...

In [20]:
#7.UPSRTC

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=1437&toCityId=733&fromCityName=Bareilly%20(Uttar%20Pradesh)&toCityName=Delhi&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=1437&toCityId=733&fromCityName=Bareilly%20(Uttar%20Pradesh)&toCityName=Delhi&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Bareilly-to-Delhi
1
Processing route: Aligarh-(uttar-pradesh)-to-Delhi
2
Processing route: Delhi-to-Bareilly
3
Processing route: Delhi-to-Aligarh-(uttar-pradesh)
4
Processing route: Farrukhabad-(Uttar-Pradesh)-to-Delhi
5
Processing route: Badaun-to-Delhi
6
Processing route: Lucknow-to-Allahabad
7
Processing route: Lucknow-to-Varanasi
8
Processing route: Delhi-to-Badaun
9
Processing route: Agra-to-Bareilly
[{'routename': 'Bareilly-to-Delhi', 'routelink': 'https://www.redbus.in/bus-tickets/Bareilly-to-Delhi?fromCityId=1437&toCityId=733&fromCityName=Bareilly%20(Uttar%20Pradesh)&toCityName=Delhi&busType=Any&srcCountry=IND&destCountry=IND&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'Vijay Tour and Travels', 'bus_type': 'A/C Seater / Sleeper (2+1)', 'dep_time': '17:00', 'd_duration': '06h 20m', 'arr_time': '23:20', 'f_fare': 'INR 649', 's_seat': '28 Seats available'}, {'routename': 'Bareilly-to-Delhi', 'routelink': 'https://www.redbus.in/bus-tickets/Bareilly

True

In [22]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

            route_name                                         route_link  \
0          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                ...                                                ...   
1212   Delhi-to-Badaun  https://www.redbus.in/bus-tickets/Delhi-to-Bad...   
1213  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1214  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1215  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1216  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   

                                                busname  \
0     Kadamba Tr

In [23]:
#8.Assam State Transport Corporation (ASTC)

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/astc')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/astc')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=90883&toCityId=74701&fromCityName=Tezpur&toCityName=Guwahati&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=90883&toCityId=74701&fromCityName=Tezpur&toCityName=Guwahati&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Tezpur-to-Guwahati
1
Processing route: Guwahati-to-Tezpur
2
Processing route: Nagaon-(Assam)-to-Guwahati
3
Processing route: Guwahati-to-Nagaon-(Assam)
4
Processing route: Goalpara-to-Guwahati
5
Processing route: Jorhat-to-North-Lakhimpur
6
Processing route: Dhubri-to-Guwahati
7
Processing route: Guwahati-to-Dhubri
8
Processing route: North-Lakhimpur-to-Sibsagar
9
Processing route: North-Lakhimpur-to-Jorhat
[]
data inserted


True

In [25]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

            route_name                                         route_link  \
0          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                ...                                                ...   
1212   Delhi-to-Badaun  https://www.redbus.in/bus-tickets/Delhi-to-Bad...   
1213  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1214  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1215  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1216  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   

                                                busname  \
0     Kadamba Tr

In [26]:
#9.Sikkim Nationalised Transport (SNT)

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/sikkim-nationalised-transport-snt')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=74709&toCityId=74694&fromCityName=Gangtok&toCityName=Siliguri&busType=Any&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=74709&toCityId=74694&fromCityName=Gangtok&toCityName=Siliguri&busType=Any&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Gangtok-to-Siliguri
1
Processing route: Siliguri-to-Gangtok
2
Processing route: Siliguri-to-Rangpo
3
Processing route: Rangpo-to-Siliguri
4
Processing route: Siliguri-to-Pelling-(Sikkim)
5
Processing route: Pelling-(Sikkim)-to-Siliguri
6
Processing route: Ravangla-(Sikkim)-to-Siliguri
7
Processing route: Siliguri-to-Jorethang-(Sikkim)
8
Processing route: Singtham-(Sikkim)-to-Siliguri
9
Processing route: Siliguri-to-Singtham-(Sikkim)
[]
data inserted


True

In [28]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

            route_name                                         route_link  \
0          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4          Goa-to-Pune  https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                ...                                                ...   
1212   Delhi-to-Badaun  https://www.redbus.in/bus-tickets/Delhi-to-Bad...   
1213  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1214  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1215  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1216  Agra-to-Bareilly  https://www.redbus.in/bus-tickets/Agra-to-Bare...   

                                                busname  \
0     Kadamba Tr

In [29]:
#10.BSRTC Operated By VIP Travels

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/bsrtc-operated-by-vip-travels')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=1603&toCityId=74699&fromCityName=Kathmandu&toCityName=Patna&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=1603&toCityId=74699&fromCityName=Kathmandu&toCityName=Patna&busType=Any&srcCountry=IND&destCountry=IND&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

# bus_lst_str = str(bus_lst)
# with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
#     write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        
    user='root',    
    password='12345678',
    database='redbus' 
)
cursor = connection.cursor()
insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
bus_data = bus_lst
for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()

0
Processing route: Kathmandu-to-Patna-(Bihar)
[{'routename': 'Kathmandu-to-Patna-(Bihar)', 'routelink': 'https://www.redbus.in/bus-tickets/Kathmandu-to-Patna-(Bihar)?fromCityId=1603&toCityId=74699&fromCityName=Kathmandu&toCityName=Patna&busType=Any&srcCountry=IND&destCountry=IND&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'V.I.P Travels', 'bus_type': 'A/C Seater Hi-Tech Push Back (2+2)', 'dep_time': '18:30', 'd_duration': '18h 00m', 'arr_time': '12:30', 'f_fare': '1040', 's_seat': '43 Seats available'}]
data inserted


True

In [31]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                      route_name  \
0                    Goa-to-Pune   
1                    Goa-to-Pune   
2                    Goa-to-Pune   
3                    Goa-to-Pune   
4                    Goa-to-Pune   
...                          ...   
1213            Agra-to-Bareilly   
1214            Agra-to-Bareilly   
1215            Agra-to-Bareilly   
1216            Agra-to-Bareilly   
1217  Kathmandu-to-Patna-(Bihar)   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                                                 ...   
1213  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1214  https://www.redbus.in/bus-tickets/Agra-to-Bare...   
1215  https://www.redbus.in/bus-tick

In [32]:
#11.Bihar state road transport corporation (BSRTC)

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# import streamlit as st

def scroll_to_bottom(driver):
    """Scroll to the bottom of the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
time.sleep(5)
scroll_to_bottom(driver)

bus_lst = []

routes = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
routes_count = len(routes)


for index in range(routes_count):
    print(index)
    driver.get('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile')
    time.sleep(5)
    scroll_to_bottom(driver)
    route_names = driver.find_elements(By.CSS_SELECTOR, "a[class=route]")
    # if index >= len(routes):
    #     print(f"Route index {index} out of range after refresh.")
    #     break

    route = route_names[index]
    route_text = route.text.replace(" ", "-")
    print(f"Processing route: {route_text}")
    travel_date = "18-Jul-2024"
    driver.get(f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward={travel_date}")
    route_url = (f"https://www.redbus.in/bus-tickets/{route_text}?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward={travel_date}")
    time.sleep(5)
    scroll_to_bottom(driver)

    bus_names = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
    bus_types = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    dep_times = driver.find_elements(By.XPATH,"//div[@class='dp-time f-19 d-color f-bold']")
    arr_times=driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
    durations=driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
    fares=driver.find_elements(By.XPATH,"//div[@class='fare d-block']")
    seats=driver.find_elements(By.XPATH,"//div[@class='seat-left m-top-30']")
    
    for busname,bustype,deptime,arrtime,duration,fare,seat in zip(
        bus_names, bus_types, dep_times, arr_times, durations, fares, seats):
        bus_dct = {
            "routename":route_text,
            "routelink":route_url,
            "t_date": travel_date,
            "bus_name": busname.text,
            "bus_type": bustype.text,
            "dep_time": deptime.text,
            "d_duration": duration.text,
            "arr_time": arrtime.text,
            "f_fare": fare.text,
            "s_seat": seat.text
            }
        bus_lst.append(bus_dct)
    
    
print(bus_lst)

bus_lst_str = str(bus_lst)
with open("/Users/nsbenakanakonda/Desktop/test.txt", 'w') as write_file:
    write_file.write(bus_lst_str)

connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

cursor = connection.cursor()

#create_table_query = """
        #CREATE TABLE redbus.bus_data_info (
            #id INT AUTO_INCREMENT PRIMARY KEY,
            #route_name VARCHAR(255),
            #route_link VARCHAR(255),
            #busname VARCHAR(255),
            #bustype VARCHAR(255),
            #departing_time VARCHAR(255),
            #duration VARCHAR(255),
            #Date VARCHAR(255),
            #reaching_time VARCHAR(255),
            #star_rating VARCHAR(255),
            #price VARCHAR(255),
            #eats_available VARCHAR(255)
#cursor.execute(create_table_query)

#print("table created")


insert_query = """
Insert INTO bus_data_info(route_name, route_link, Date, busname, bustype, departing_time, duration, reaching_time, price, seats_available)
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

bus_data = bus_lst

for record in bus_data:
    cursor.execute(insert_query,
                   (
                       record["routename"],
                       record["routelink"],
                       record["t_date"],
                       record["bus_name"],
                       record["bus_type"],
                       record["dep_time"],
                       record["d_duration"],
                       record["arr_time"],
                       record["f_fare"],
                       record["s_seat"]
                       )
        )

connection.commit()

print("data inserted")

cursor.close()
connection.close()


0
Processing route: Patna-(Bihar)-to-Bettiah
1
Processing route: Gopalganj-(Bihar)-to-Delhi
2
Processing route: Patna-(Bihar)-to-Motihari
3
Processing route: Delhi-to-Motihari
4
Processing route: Bettiah-to-Patna-(Bihar)
5
Processing route: Motihari-to-Delhi
6
Processing route: Patna-(Bihar)-to-Balmiki-Nagar-(bihar)
7
Processing route: Balmiki-Nagar-(bihar)-to-Patna-(Bihar)
8
Processing route: Patna-(Bihar)-to-Kathmandu
9
Processing route: Patna-(Bihar)-to-Katihar
[{'routename': 'Patna-(Bihar)-to-Bettiah', 'routelink': 'https://www.redbus.in/bus-tickets/Patna-(Bihar)-to-Bettiah?fromCityId=130&toCityId=210&fromCityName=Pune&toCityName=Goa&busType=Any&onward=18-Jul-2024', 't_date': '18-Jul-2024', 'bus_name': 'Kadamba Transport Corporation Limited (KTCL) - 89821', 'bus_type': 'A/C Sleeper (2+1)', 'dep_time': '19:15', 'd_duration': '10h 35m', 'arr_time': '05:50', 'f_fare': 'INR 800', 's_seat': '17 Seats available'}, {'routename': 'Patna-(Bihar)-to-Bettiah', 'routelink': 'https://www.redbus

In [34]:
import mysql.connector
import streamlit as st
import pandas as pd

# Establish connection to MySQL
connection = mysql.connector.connect(
    host='localhost',        # e.g., 'localhost' or IP address
    user='root',    # e.g., 'root'
    password='12345678',# e.g., 'password'
    database='redbus' # e.g., 'test_db'
)

# Function to fetch data from MySQL
def fetch_data():
    cursor = connection.cursor(dictionary=True)
    query = """
    SELECT route_name, route_link, busname, bustype, departing_time, duration, Date, reaching_time, star_rating, price, seats_available
    FROM bus_data_info
    """
    cursor.execute(query)
    data = cursor.fetchall()
    cursor.close()
    return data

# Fetch data from MySQL
bus_data = fetch_data()

df = pd.DataFrame(bus_data)

print(df)
# Streamlit app
st.title("Bus Route Information")

col1, col2, col3 = st.columns(3)
# Create dropdowns for routelink, d_date, bus_name, bus_type
with col1:
    selected_route_name = st.selectbox("Select Route Link", sorted(set(df['route_name'] )))
with col2:
    selected_d_date = st.selectbox("Select Travel Date", sorted(set(df['Date'])))
with col3:
    selected_bus_type = st.selectbox("Select Bus Type", sorted(set(df['bustype'])))

# Filter data based on selected dropdown values
filtered_data = df

# st.table(filtered_data)
if selected_route_name:
    filtered_data = filtered_data[filtered_data['route_name'] == selected_route_name]
if selected_d_date:
    filtered_data = filtered_data[filtered_data['Date'] == selected_d_date]
if selected_bus_type:
    filtered_data = filtered_data[filtered_data['bustype'] == selected_bus_type]

# Display filtered results in a table format
if not filtered_data.empty:
    st.subheader("Results:")
    st.table(filtered_data)
    
else:
    st.warning("No data found for selected filters.")

                    route_name  \
0                  Goa-to-Pune   
1                  Goa-to-Pune   
2                  Goa-to-Pune   
3                  Goa-to-Pune   
4                  Goa-to-Pune   
...                        ...   
1263  Patna-(Bihar)-to-Katihar   
1264  Patna-(Bihar)-to-Katihar   
1265  Patna-(Bihar)-to-Katihar   
1266  Patna-(Bihar)-to-Katihar   
1267  Patna-(Bihar)-to-Katihar   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
1     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
2     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
3     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
4     https://www.redbus.in/bus-tickets/Goa-to-Pune?...   
...                                                 ...   
1263  https://www.redbus.in/bus-tickets/Patna-(Bihar...   
1264  https://www.redbus.in/bus-tickets/Patna-(Bihar...   
1265  https://www.redbus.in/bus-tickets/Patna-(Bihar...   
1